<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/label_export/video.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/label_export/video.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Video Data Export
Export labels from video annotation projects.

In [ ]:
!pip install -q labelbox numpy matplotlib ipython

In [ ]:
from labelbox import Client
from matplotlib import pyplot as plt
from IPython.display import clear_output
import numpy as np
import json
import ndjson
import requests
import cv2
from typing import Dict, Any
import os

In [ ]:
# Pick a video project with completed bounding box labels
PROJECT_ID = ""

# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your api key
API_KEY = None
client = Client(api_key=API_KEY)
project = client.get_project(PROJECT_ID)

### Export the labels

In [ ]:
export_url = project.export_labels()

# labels can also be exported with `start` and `end` filters
# export_url = project.export_labels(start="2020-01-01", end="2020-01-02")

In [ ]:
print(export_url)

https://storage.googleapis.com/labelbox-exports/cktu4ft7e3xvp0y6tgv24d9mw/cktu4ft8q3xvy0y6t41p5dh9g/export-2022-04-07T14%3A31%3A30.021Z.json?GoogleAccessId=api-prod%40labelbox-193903.iam.gserviceaccount.com&Expires=1650551494&Signature=T73XqzvQlFyzMJ9tGrcoGxON8nJjXluBNpcGB0rmvMPnbaWBxcAaC%2Fe7GGltHauvXZyrjaO8fBj3KlLuPFwqcQXxOF9R1OtAwq1b6gB4VCbuls4N%2BsxJG1CQ6WxKE5nV8pRf9lEl839YSc7j2TY6agOFWmIvkj4FTKzK%2F2ezwq9KH8YZCU%2FhVVrvEnjKM5phDcCNvQqbthmr84bj9DnTe24aOML7qKrjsV8mRlOi8PDoOy4orNSxk%2FWfRNRw4Gck9Q4avbzS6XW16RMRcFKqBIIhBEImlhfb0yr9ZAI2vi5CK6u3eVwXwtXD%2BfIfZPcxxDHs%2FE%2B7LGvg%2BUGQoCjHSA%3D%3D&response-content-disposition=attachment


In [ ]:
exports = requests.get(export_url).json()

To get more information on the fields in the label payload, follow [our documentation here](https://docs.labelbox.com/data-model/en/index-en#label).

In [ ]:
# Note that for a video export, a list of exports is created, with one export per data row
type(exports)

list

In [ ]:
# Let's check out the export for the first data row (video)
exports[0]

{'ID': 'cktu4ftfi3y1k0y6t870p45ty',
 'DataRow ID': 'cktu4ft9q3xz30y6t3z25hymy',
 'Labeled Data': 'https://storage.labelbox.com/cjhfn5y6s0pk507024nz1ocys%2Fb8837f3b-b071-98d9-645e-2e2c0302393b-jellyfish2-100-110.mp4?Expires=1650551493520&KeyName=labelbox-assets-key-3&Signature=RrxYxKqAP0VbN0ZiJMeJ2yhzpyI',
 'Label': {'objects': [],
  'classifications': [],
  'relationships': [],
  'frames': 'https://api.labelbox.com/v1/frames/cktu4ftfi3y1k0y6t870p45ty'},
 'Created By': 'eemerson@labelbox.com',
 'Project Name': 'Sample Video Project',
 'Created At': '2021-09-21T13:36:51.000Z',
 'Updated At': '2021-09-21T13:36:51.412Z',
 'Seconds to Label': 90.22,
 'External ID': 'jellyfish2-100-110.mp4',
 'Agreement': -1,
 'Benchmark Agreement': -1,
 'Benchmark ID': None,
 'Dataset Name': 'Example Jellyfish Dataset',
 'Reviews': [],
 'View Label': 'https://editor.labelbox.com?project=cktu4ft8q3xvy0y6t41p5dh9g&label=cktu4ftfi3y1k0y6t870p45ty',
 'Has Open Issues': 0,
 'Skipped': False}

In [ ]:
# View some specific fields of the export
print("Label ID:", exports[0]['DataRow ID'])
print("Created By:", exports[0]['Created By'])
print("Created At:", exports[0]['Created At'])
print("Reviews:", exports[0]['Reviews'])

Label ID: cktu4ft9q3xz30y6t3z25hymy
Created By: eemerson@labelbox.com
Created At: 2021-09-21T13:36:51.000Z
Reviews: []


In [ ]:
# View the video in your browser by clicking on the link produced
video_url = exports[0]["Labeled Data"]
print(video_url)

https://storage.labelbox.com/cjhfn5y6s0pk507024nz1ocys%2Fb8837f3b-b071-98d9-645e-2e2c0302393b-jellyfish2-100-110.mp4?Expires=1650551493520&KeyName=labelbox-assets-key-3&Signature=RrxYxKqAP0VbN0ZiJMeJ2yhzpyI


### View the annotation data
The label for each frame of the video must be fetched individually.

In [ ]:
# Grab the annotations url
annotations_url = exports[0]["Label"]["frames"]
print(annotations_url)

https://api.labelbox.com/v1/frames/cktu4ftfi3y1k0y6t870p45ty


In [ ]:
# Provide the appropriate authorization to view the labeled frames
headers = {"Authorization": f"Bearer {API_KEY}"}
annotations = ndjson.loads(requests.get(annotations_url, headers=headers).text)

In [ ]:
# Make the data easy to lookup by assigning each frame its frame number
annotations = {annot["frameNumber"]: annot for annot in annotations}

# Grab the first frame and print the contents
first_frame = annotations[1]
print(json.dumps(first_frame, indent=2))

{
  "frameNumber": 1,
  "classifications": [],
  "objects": [
    {
      "featureId": "cktu4ftf43y1c0y6t7r8oc7tr",
      "schemaId": "cktu4ftcs3y0h0y6t1bftfa1f",
      "title": "Jellyfish",
      "value": "jellyfish",
      "color": "#a23030",
      "keyframe": true,
      "bbox": {
        "top": 582,
        "left": 1644,
        "height": 340,
        "width": 212
      },
      "classifications": []
    },
    {
      "featureId": "cktu4ftf43y1d0y6t14xzhymp",
      "schemaId": "cktu4ftcs3y0h0y6t1bftfa1f",
      "title": "Jellyfish",
      "value": "jellyfish",
      "color": "#a23030",
      "keyframe": true,
      "bbox": {
        "top": 712,
        "left": 1256,
        "height": 204,
        "width": 330
      },
      "classifications": []
    },
    {
      "featureId": "cktu4ftf43y1e0y6tgh0y78om",
      "schemaId": "cktu4ftcs3y0h0y6t1bftfa1f",
      "title": "Jellyfish",
      "value": "jellyfish",
      "color": "#a23030",
      "keyframe": true,
      "bbox": {
        "

In [ ]:
# Grab values of the first object in the first annotation
print("schemaId:", first_frame['objects'][0]['schemaId'])
print("title:", first_frame['objects'][0]['title'])
print("is a keyframe?:", first_frame['objects'][0]['keyframe'])
print("bbox dimensions:", first_frame['objects'][0]['bbox'])

schemaId: cktu4ftcs3y0h0y6t1bftfa1f
title: Jellyfish
is a keyframe?: True
bbox dimensions: {'top': 582, 'left': 1644, 'height': 340, 'width': 212}


### Visualize annotations

Turn video into individual frames

In [ ]:
# Store the video url in a file
with open("/tmp/sample_video.mp4", "wb") as file:
    file.write(requests.get(video_url).content)

In [ ]:
vidcap = cv2.VideoCapture("/tmp/sample_video.mp4")
success, image = vidcap.read()
image = image[:, :, ::-1]
# Note that frameNumber 1 in the annotation is frame index 0s
count = 1
while success and count < 20:
    plt.figure(1)
    plt.imshow(image)
    plt.title('frameNumber ' + str(count))
    plt.pause(0.25)
    plt.clf()
    success, image = vidcap.read()
    count += 1
    if success and count < 20:
        clear_output(wait=True)
    image = image[:, :, ::-1]

In [ ]:
# Helper function
def visualize_bbox(image: np.ndarray, tool: Dict[str, Any]) -> np.ndarray:
    """
    Draws a bounding box on an image
    
    Args:
        image (np.ndarray): image to draw a bounding box onto
        tool (Dict[str,any]): Dict response from the export
    Returns:
        image with a bounding box drawn on it.
    """
    start = (int(tool["bbox"]["left"]), int(tool["bbox"]["top"]))
    end = (int(tool["bbox"]["left"] + tool["bbox"]["width"]),
           int(tool["bbox"]["top"] + tool["bbox"]["height"]))
    return cv2.rectangle(image, start, end, (255, 0, 0), 5)

Overlay annotations on frames

In [ ]:
vidcap = cv2.VideoCapture('/tmp/sample_video.mp4')
success, image = vidcap.read()
image = image[:, :, ::-1]
# Note that frameNumber 1 in the annotation is frame index 0
count = 1

while success and count < 20:
    annotation = annotations.get(count)
    if annotation is not None:
        for tool in annotation['objects']:
            if 'bbox' in tool:
                image = visualize_bbox(image.astype(np.uint8), tool)
    plt.figure(1)
    plt.imshow(image)
    plt.title('frameNumber ' + str(count))
    plt.pause(0.25)
    plt.clf()

    success, image = vidcap.read()
    count += 1
    if success and count < 20:
        clear_output(wait=True)
    image = image[:, :, ::-1]